In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt


# Importing important libraries

In [2]:
import math
import seaborn as sns
import plotly.express as px
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge

# Loading the training dataset

In [3]:
#Reading the training file
df_train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df_train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [4]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# Data Preprocessing:

## Checking and removal of null values

In [17]:
for i in df_train.columns:
    print( i+" \t: " +str(df_train[i].isnull().sum()))

Id 	: 0
MSSubClass 	: 0
MSZoning 	: 0
LotFrontage 	: 0
LotArea 	: 0
Street 	: 0
Alley 	: 0
LotShape 	: 0
LandContour 	: 0
Utilities 	: 0
LotConfig 	: 0
LandSlope 	: 0
Neighborhood 	: 0
Condition1 	: 0
Condition2 	: 0
BldgType 	: 0
HouseStyle 	: 0
OverallQual 	: 0
OverallCond 	: 0
YearBuilt 	: 0
YearRemodAdd 	: 0
RoofStyle 	: 0
RoofMatl 	: 0
Exterior1st 	: 0
Exterior2nd 	: 0
MasVnrType 	: 0
MasVnrArea 	: 0
ExterQual 	: 0
ExterCond 	: 0
Foundation 	: 0
BsmtQual 	: 0
BsmtCond 	: 0
BsmtExposure 	: 0
BsmtFinType1 	: 0
BsmtFinSF1 	: 0
BsmtFinType2 	: 0
BsmtFinSF2 	: 0
BsmtUnfSF 	: 0
TotalBsmtSF 	: 0
Heating 	: 0
HeatingQC 	: 0
CentralAir 	: 0
Electrical 	: 0
1stFlrSF 	: 0
2ndFlrSF 	: 0
LowQualFinSF 	: 0
GrLivArea 	: 0
BsmtFullBath 	: 0
BsmtHalfBath 	: 0
FullBath 	: 0
HalfBath 	: 0
BedroomAbvGr 	: 0
KitchenAbvGr 	: 0
KitchenQual 	: 0
TotRmsAbvGrd 	: 0
Functional 	: 0
Fireplaces 	: 0
FireplaceQu 	: 0
GarageType 	: 0
GarageYrBlt 	: 0
GarageFinish 	: 0
GarageCars 	: 0
GarageArea 	: 0
GarageQual 

As we can see there are a lot of null values so we need to replace these null values.

### Defining what to replace with.

In [18]:
max_replacements = ['MSZoning','Utilities','Exterior1st','Exterior2nd','KitchenQual',
                    'Functional','MasVnrType','Electrical','SaleType']

median_replacements=['LotFrontage','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF',
                     'BsmtFullBath','BsmtHalfBath','GarageCars','GarageArea',
                     'MasVnrArea']

na_replacements = ['Alley','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1',
                   'BsmtFinType2','FireplaceQu','GarageType','GarageFinish',
                   'GarageQual','GarageCond','PoolQC','Fence','MiscFeature']

mean_replacements = ['GarageYrBlt']

Max: It is for all the categoricla data where we are replacing the null with most common occuring data in that column.

Median: It is for all the fields where the data is numeric and has some sort of order to it but also is either categorical or has some outliers.(Although we could have used mean here, the mean is affected by alot by outliers (For decinding this, the box plots have been dravn later.))

NA: It is for those data fields where NA is a separate category as mentioned in the data_description file and hence we have to consider it as an individual field.

Mean: It is for numeric fields that do not have many outliers. 

P.S: We have only defined replacements for the fields with missing values in either test or train sets.

### Applying the replacements. 

In [19]:
for i in max_replacements:
    value = df_train[i].value_counts().idxmax()
    df_train[i] = df_train[i].fillna(value)
    
for i in median_replacements:
    value = df_train[i].median()
    df_train[i] = df_train[i].fillna(value)
    
for i in na_replacements:
    value = 'NA'
    df_train[i] = df_train[i].fillna(value)
    
for i in mean_replacements:
    value = df_train[i].mean()
    df_train[i] = df_train[i].fillna(value)

### Verifying if all null values are removed. 

In [20]:
print('Total no. of null values now are : '+ str(df_train[i].isnull().sum().sum()))

Total no. of null values now are : 0


## Separating categorical and continuous data fields

In [21]:
list_cat = ['MSSubClass','MSZoning','Street','Alley', 'LotShape', 'LandContour',
            'Utilities', 'LotConfig','LandSlope', 'Neighborhood','Condition1', 
            'Condition2','BldgType','HouseStyle', 'OverallQual','OverallCond',
            'RoofStyle', 'RoofMatl', 'Exterior1st','Exterior2nd', 'MasVnrType',
            'ExterQual','ExterCond', 'Foundation', 'BsmtQual','BsmtCond',
            'BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC', 
            'CentralAir', 'Electrical','BsmtFullBath','BsmtHalfBath','FullBath',
            'HalfBath','BedroomAbvGr', 'KitchenAbvGr','KitchenQual',
            'TotRmsAbvGrd','Functional','Fireplaces','FireplaceQu','GarageType',
            'GarageFinish','GarageCars','GarageQual','GarageCond', 'PavedDrive',
            'PoolQC','Fence','MiscFeature','SaleType','SaleCondition']

list_cont = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd','MasVnrArea',
             'BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF',
             '2ndFlrSF', 'LowQualFinSF', 'GrLivArea','GarageYrBlt','GarageArea',
             'WoodDeckSF', 'OpenPorchSF','EnclosedPorch', '3SsnPorch', 
             'ScreenPorch', 'PoolArea','MiscVal', 'MoSold', 'YrSold',]

print('No. of columns with categorical data values are : '+str(len(list_cat)))

print('No. of columns with continuous data values are : '+str(len(list_cont)))

No. of columns with categorical data values are : 55
No. of columns with continuous data values are : 24


List_cate: It is the list of all the categorical data fields in the dataset.

List_cont: It is the list of all the continuous data fields in the dataset.(except SalePrice)

In [22]:
df_train.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Id_visual
0,1,60,RL,65.0,8450,Pave,NA,Reg,Lvl,AllPub,...,NA,NA,NA,0,2,2008,WD,Normal,208500,ID : 1
1,2,20,RL,80.0,9600,Pave,NA,Reg,Lvl,AllPub,...,NA,NA,NA,0,5,2007,WD,Normal,181500,ID : 2
2,3,60,RL,68.0,11250,Pave,NA,IR1,Lvl,AllPub,...,NA,NA,NA,0,9,2008,WD,Normal,223500,ID : 3
3,4,70,RL,60.0,9550,Pave,NA,IR1,Lvl,AllPub,...,NA,NA,NA,0,2,2006,WD,Abnorml,140000,ID : 4
4,5,60,RL,84.0,14260,Pave,NA,IR1,Lvl,AllPub,...,NA,NA,NA,0,12,2008,WD,Normal,250000,ID : 5
5,6,50,RL,85.0,14115,Pave,NA,IR1,Lvl,AllPub,...,NA,MnPrv,Shed,700,10,2009,WD,Normal,143000,ID : 6
6,7,20,RL,75.0,10084,Pave,NA,Reg,Lvl,AllPub,...,NA,NA,NA,0,8,2007,WD,Normal,307000,ID : 7
7,8,60,RL,69.0,10382,Pave,NA,IR1,Lvl,AllPub,...,NA,NA,Shed,350,11,2009,WD,Normal,200000,ID : 8
8,9,50,RM,51.0,6120,Pave,NA,Reg,Lvl,AllPub,...,NA,NA,NA,0,4,2008,WD,Abnorml,129900,ID : 9
9,10,190,RL,50.0,7420,Pave,NA,Reg,Lvl,AllPub,...,NA,NA,NA,0,1,2008,WD,Normal,118000,ID : 10


# Data Visualisation: 

## Box Plot of the sale price over the whole dataset.

In [10]:
column = 'Id'
name = 'ID'
a= []
for i in df_train.index:
    a.append(name +' : '+ str(df_train[column][i]))
df_train[column+'_visual'] = a

In this we have added an extra visual column for the box plot.

In [11]:
fig = px.box(data_frame = df_train.reset_index(),hover_name = 'Id_visual',
             y = 'SalePrice',hover_data = ['MoSold', 'YrSold'],height = 500,
             width = 400,labels = {'SalePrice':'Sale Price in "$"'},
             title = 'Box plot of the sale price(Hover for details)')
fig.show()

As we can see from the box plot there are a lot of outliers so we set a threshold for the price. Here i have taken it to be $450000

In [12]:
removed = 0
threshold = 450000
for i in df_train.index:
    if df_train['SalePrice'][i]>threshold:
        df_train = df_train.drop(i)
        removed+=1
print('Total data points removed till now are: '+str(removed))

Total data points removed till now are: 14


Since we  do not want to remove too many data ponts, we will keep a track of how many data points have veen removed up till any step.

In [ ]:
dic1 = {}
for i in list_cat :
    a = []
    for j in file.groupby(i).mean().index:
        a.append(j)
    dic1[i] = a
dic1['Alley'] = ['Grvl','Pave','NA']
dic1['LandContour'] = ['Lvl','Bnk', 'HLS', 'Low']
dic1['LotConfig'] = ['Inside','Corner', 'CulDSac', 'FR2', 'FR3']
dic1['ExterQual'] = ['Ex', 'Gd', 'TA', 'Fa']
dic1['ExterCond'] = ['Ex','Gd', 'TA','Fa','Po']
dic1['BsmtQual'] = ['Ex', 'Gd', 'TA', 'Fa','NA']
dic1['BsmtCond'] = ['Gd', 'TA', 'Fa','Po','NA']
dic1['BsmtExposure'] = ['Gd','Av', 'Mn', 'No','NA']
dic1['BsmtFinType1'] = ['GLQ','ALQ', 'BLQ','Rec', 'LwQ', 'Unf','NA']
dic1['BsmtFinType2'] = ['GLQ','ALQ', 'BLQ','Rec', 'LwQ', 'Unf','NA']
dic1['HeatingQC'] = ['Ex','Gd', 'TA','Fa','Po']
dic1['Electrical'] = ['SBrkr','FuseA', 'FuseF', 'FuseP', 'Mix']
dic1['KitchenQual'] = ['Ex','Gd', 'TA','Fa']
dic1['Functional']=['Typ','Min1', 'Min2', 'Mod','Maj1', 'Maj2', 'Sev']
dic1['FireplaceQu'] = ['Ex','Gd', 'TA','Fa','Po','NA']
dic1['GarageQual'] =['Ex','Gd', 'TA','Fa','Po','NA']
dic1['GarageCond'] =['Ex','Gd', 'TA','Fa','Po','NA']
dic1['GarageFinish'] = ['Fin','RFn','Unf','NA']
dic1['PoolQC'] = ['Ex','Gd','Fa','NA']
dic1['Fence'] = ['GdPrv','MnPrv','GdWo', 'MnWw','NA']
dic1['SaleType'] = ['WD','CWD','New','COD', 'Con','ConLw', 'ConLI','ConLD','Oth', ]
dic1['SaleCondition'] = ['Normal','Abnorml', 'AdjLand', 'Alloca', 'Family','Partial']

In [ ]:
y = 'SalePrice'
n = 3
s= 20
f,axes = plt.subplots(19,n,figsize = (s,6*s))
counter = 0
for i in list_cat:
    sns.barplot(x = i , y = y , data  = file,order= dic1[i], ax = axes[counter//n][counter%n],saturation = 1)
    counter+=1

In [ ]:
df = file.groupby('Alley').mean()
df

In [ ]:
z = 1.960 #confidence level of 95% (for 99% use 3.291)
s = 'Alley'
con = []
sale_vis = []
count = []
for i in df.index:
    a = []
    counter = 0
    for j in file.index:
        if file[s][j] == i:
            a.append(file['SalePrice'][j]-df['SalePrice'][i])
            counter+=1
    count.append(counter)
    std = np.std(a)
    confidence = std/(math.sqrt(counter))
    con.append((z*confidence)//1)
    sale_vis.append('Sale Price : ' + str(df['SalePrice'][i]//1))
df ['Confidence'] = con 
df ['sale_vis'] = sale_vis
df ['Total Count'] = count
count_per = []
for i in df.index:
    per = df['Total Count'][i]/np.sum(count)
    per = (per*10000)//1
    per= per//100
    count_per.append(str(per)+'%')
df['Count Percent'] = count_per
df

In [ ]:
dic1['Alley'] = ['Grvl','Pave','NA']
i = 'Alley'

fig = px.bar(data_frame = df.reset_index(),y='SalePrice', color = i ,x = i,category_orders = dic1,error_y = 'Confidence',
             hover_name = sale_vis,hover_data = ['Total Count','Count Percent'],opacity= 1,labels = {y : 'Sale Price in "$"', 'Grvl':'Gravel','Pave':'Paved','NA':'Not Paved'})
fig.show()

In [ ]:
#List of features on which to apply one hot encoding before applying regression:
list_pure_categorical = ['MSSubClass','MSZoning','LotShape','LandContour', 'LotConfig', 'Neighborhood','Condition1', 'Condition2', 'BldgType',
                        'HouseStyle','RoofStyle', 'RoofMatl','Exterior1st','Exterior2nd','Foundation','Heating','GarageType','SaleType',
                         'SaleCondition','MiscFeature','MasVnrType']

#List of categorical features that we can directly apply regression on:
categorical_ordered = ['Street','Alley','Utilities','LandSlope','OverallQual', 'OverallCond','ExterQual', 'ExterCond','BsmtQual','BsmtCond',
                       'BsmtExposure','BsmtFinType1','BsmtFinType2','HeatingQC', 'CentralAir','Electrical','KitchenQual','BsmtFullBath', 
                       'BsmtHalfBath', 'FullBath','HalfBath','BedroomAbvGr', 'KitchenAbvGr','TotRmsAbvGrd','Functional','Fireplaces',
                       'FireplaceQu','GarageFinish', 'GarageCars','GarageQual','GarageCond', 'PavedDrive','PoolQC','Fence']

#List of continuous variables:
list_continuous = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','TotalBsmtSF','1stFlrSF',
                   'GarageYrBlt', 'GarageArea','WoodDeckSF', 'OpenPorchSF','EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea','MiscVal',
                   'MoSold', 'YrSold','LowQualFinSF', 'GrLivArea','2ndFlrSF']

In [ ]:
file.head()

In [ ]:
for i in categorical_ordered:
    a = []
    for j in file.index:
        for k in range(len(dic1[i])):
            if file[i][j] == dic1[i][k]:
                a.append(k+1)
    file[i] = a

In [ ]:
file.head()

In [ ]:
n = 3
s= 20
f,axes = plt.subplots(3*n-1,n,figsize = (s,3*s))
counter = 0
for i in list_cont:
    sns.boxplot( y = i,data  = file, ax = axes[counter//n][counter%n])
    counter+=1

In [ ]:
outlier ={'LotFrontage':150, 'LotArea':100000,'MasVnrArea':900,'BsmtFinSF1':2000,'TotalBsmtSF':2500,
               '1stFlrSF':2500, 'GarageArea':1130,'WoodDeckSF':600, 'OpenPorchSF':310,
               'EnclosedPorch':310,'3SsnPorch':350,'MiscVal':6000,'GrLivArea':3500, 'BsmtFullBath':2.5,'2ndFlrSF':1750}
for j in outlier:
    for i in file.index:
        if file[j][i]>outlier[j]:
            file = file.drop(i)
            removed+=1
    print(j + ' : '+ str(removed))
for i in file.index:
    if file['YearBuilt'][i]<1880:
        file = file.drop(i)
        removed+=1
print('YearBuilt' + ' : '+ str(removed))

In [ ]:
y = 'SalePrice'
n = 3
s= 20
f,axes = plt.subplots(8,n,figsize = (s,3*s),sharey=True)
counter = 0
for i in list_continuous:
    sns.lineplot(x = i , y = y , data  = file, ax = axes[counter//n][counter%n])
    counter+=1

In [ ]:
corr  = file[categorical_ordered+list_continuous + ['SalePrice']].corr()
f,axes = plt.subplots(1,1,figsize = (35,35))
sns.heatmap(corr,square=True,annot = True,linewidth = .1,linecolor = 'black',ax = axes,center = 1.4)

In [ ]:
columns = categorical_ordered+list_continuous + ['SalePrice']
useful = []
for i in columns:
    if (corr[i]['SalePrice'])>=.15 or (corr[i]['SalePrice'])<=-.15:
        useful.append(i)
useful
        

columns = categorical_ordered+list_continuous + ['SalePrice']
fig = px.imshow(corr,x = columns,y = columns)
fig.show()

In [ ]:
file.columns

In [ ]:
file[useful]

In [ ]:
print(file.columns)

In [ ]:
useful_category = []
for j in list_pure_categorical:
    for i in file.groupby(j).count().index:
        s = j+str(i)
        a=[]
        for k in file.index:
            if file[j][k]==i:
                a.append(1)
            else:
                a.append(0)
        file[s]=a
        useful_category.append(s)
len (useful_category)

In [ ]:
corr  = file[useful_category + ['SalePrice']].corr()
f,axes = plt.subplots(1,1,figsize = (40,40))
sns.heatmap(corr,square=True,linewidth = .1,linecolor = 'black',ax = axes,center = 1.4)

In [ ]:
columns = useful_category + ['SalePrice']
final_useful = []
for i in columns:
    if (corr[i]['SalePrice'])>=.15 or (corr[i]['SalePrice'])<=-.15:
        final_useful.append(i)
final_useful

In [ ]:
useful = useful+final_useful
useful

In [ ]:
df_train_x = file[useful].drop(['SalePrice'],axis = 1)
df_train_y = file[['SalePrice']]
x_train, x_test, y_train, y_test = train_test_split(df_train_x, df_train_y, test_size=0.15, random_state=42)

In [ ]:
poly = PolynomialFeatures(degree=2)
poly_x_train = poly.fit_transform(x_train)
poly_x_test = poly.fit_transform(x_test)

In [ ]:
predict = []
reg = LinearRegression().fit(x_train, y_train)
pre_reg = reg.predict(x_test)

reg_poly = LinearRegression().fit(poly_x_train, y_train)
pre_reg_poly = reg_poly.predict(poly_x_test)

predict.append(pre_reg)
predict.append(pre_reg_poly)


for prediction in predict:
    pre = []
    for p in prediction:
        if p < 0:
            pre.append(0)
        else:
            pre.append(p)
    print(np.sqrt(mean_squared_log_error( y_test, pre )))

In [ ]:
reg = LinearRegression().fit(df_train_x, df_train_y)

In [ ]:
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
test.describe()

In [ ]:
for i in test.columns:
    print( i+" \t: " +str(test[i].isnull().sum()))

In [ ]:
for i in null_replacements:
    test[i] = test[i].fillna(null_replacements[i])

In [ ]:
for i in test.columns:
    print( i+" \t: " +str(test[i].isnull().sum()))

In [ ]:
useful_category = []
for j in list_pure_categorical:
    for i in file.groupby(j).count().index:
        s = j+str(i)
        a=[]
        for k in test.index:
            if test[j][k]==i:
                a.append(1)
            else:
                a.append(0)
        test[s]=a
        useful_category.append(s)
len (useful_category)

In [ ]:
test

In [ ]:
encode_list = ['Street','Alley','Utilities','LandSlope','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1',
               'BsmtFinType2','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','FireplaceQu','GarageFinish','GarageQual',
               'GarageCond','PavedDrive','PoolQC','Fence']

In [ ]:
for i in encode_list:
    a = []
    for j in test.index:
        for k in range(len(dic1[i])):
            if test[i][j] == dic1[i][k]:
                a.append(k+1)
    test[i] = a

In [ ]:
useful

In [ ]:
useful1 = []
for i in useful:
    if i != 'SalePrice':
        useful1.append(i)
useful1

In [ ]:
test[useful1]

In [ ]:
pre = reg.predict(test[useful1])
test ['SalePrice'] = pre

In [ ]:
test

In [ ]:
test[['Id','SalePrice']].describe()

In [ ]:
test[['Id','SalePrice']].to_csv('submission.csv',index=False)